<a href="https://colab.research.google.com/github/gilvandrocesardemedeiros/DIMAP-BigData/blob/master/Copy_of_Projeto_Final%5BMotor%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

# Pré-processamento e análise exploratória

In [0]:
dataBase = pd.read_excel("https://github.com/gilvandrocesardemedeiros/DIMAP-BigData/raw/master/Data/Motor_original.xls.xlsx").drop(0).reset_index(drop = True)

In [0]:
dataBase.describe()

,Km Atual,Km Troca,Fe,Cr,Pb,Cu,Sn,Al,Ni,Ag,Si,B,Na,Mg,Ca,Ba,P,Zn,Mo,Ti,V,K
count,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000,784.000000
mean,315414.098214,11084.375000,98.929847,3.748724,11.475765,7.588010,4.149235,8.303571,0.723214,0.025510,10.219388,0.207908,168.258929,473.456633,1617.765306,0.173469,888.798469,1121.316327,6.700255,-0.008929,-0.014031,4.353316
std,121579.353525,4791.935629,58.395240,3.704179,9.206829,12.164681,2.253290,4.182270,0.908972,0.400335,4.279248,2.443515,109.671904,304.924316,729.105599,2.089446,248.835398,329.102847,5.439621,0.147077,0.128085,3.704116
min,27752.000000,1619.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,252879.750000,8792.500000,67.000000,2.000000,7.000000,4.000000,3.000000,6.000000,0.000000,0.000000,8.000000,0.000000,31.000000,91.500000,1112.500000,0.000000,652.000000,813.750000,3.000000,0.000000,0.000000,1.000000
50%,291789.000000,9913.500000,85.500000,3.000000,10.000000,5.000000,4.000000,8.000000,0.000000,0.000000,10.000000,0.000000,227.000000,615.000000,1237.500000,0.000000,971.000000,1223.000000,6.000000,0.000000,0.000000,4.000000
75%,368230.000000,11634.000000,116.000000,4.000000,14.000000,7.000000,5.000000,10.000000,1.000000,0.000000,12.000000,0.000000,254.000000,723.000000,2421.500000,0.000000,1073.250000,1364.000000,9.000000,0.000000,0.000000,6.000000
max,883564.000000,52697.000000,607.000000,31.000000,127.000000,189.000000,27.000000,43.000000,5.000000,7.000000,45.000000,64.000000,400.000000,1063.000000,3828.000000,30.000000,1429.000000,2004.000000,63.000000,1.000000,1.000000,24.000000


In [0]:
dataBase.head()

,Código,Tipo,Marca,Km Atual,Km Troca,Diagnostico,Data Troca,Fe,Cr,Pb,Cu,Sn,Al,Ni,Ag,Si,B,Na,Mg,Ca,Ba,P,Zn,Mo,Ti,V,K,Oxi,Nit,Sul,Ful,Agu,Gli,Dil
0,65307:,OM 366\tMOTOR,RIMULA SMV,88898.0,14185.0,GGG,1998-09-07 00:00:00,87.0,15.0,7.0,8.0,8.0,2.0,0.0,0.0,7.0,0.0,196.0,586.0,1305.0,0.0,952.0,1182.0,0.0,0.0,0.0,4.0,"0,03","0,06","0,21","1,13","<0,10","<0,08","<2,00"
1,61818:,OM 366\tMOTOR,RIMULA SMV,85790.0,12670.0,NaN,15/06/98,97.0,19.0,7.0,24.0,10.0,2.0,0.0,0.0,7.0,0.0,231.0,719.0,1111.0,0.0,1057.0,1320.0,0.0,0.0,0.0,4.0,"0,05","0,07","0,23","1,24","<0,10","<0,08","<2,00"
2,61679:,OM 366\tMOTOR,RIMULA SMV,85817.0,17044.0,NaN,15/06/98,138.0,26.0,9.0,16.0,11.0,2.0,0.0,0.0,9.0,0.0,221.0,717.0,1135.0,0.0,1020.0,1270.0,0.0,0.0,0.0,5.0,"0,08","0,11","0,30","1,65","<0,10","<0,08","<2,00"
3,68053:,OM 366\tMOTOR,RIMULA SMV,100314.0,14805.0,GYG,27/07/98,99.0,21.0,10.0,8.0,10.0,2.0,0.0,0.0,7.0,0.0,193.0,558.0,1284.0,0.0,949.0,1177.0,0.0,0.0,0.0,5.0,"0,05","0,09","0,25","1,50","<0,10","<0,08","<2,00"
4,50253:,OM 366\tMOTOR,RIMULA SMV,436880.0,3637.0,GGG,26/01/98,146.0,5.0,13.0,5.0,6.0,15.0,1.0,0.0,15.0,0.0,242.0,676.0,1167.0,0.0,1038.0,1317.0,9.0,0.0,0.0,7.0,"0,03","0,03","0,07","0,45",<0.10,"<0,08","<2,00"


In [0]:
#Verificou-se através de uma exploração na base de dados que a ocorrência de NaN está presente em diversos diagnósticos
#Inviabilizando, assim, a análise para estes casos, necessitando que sejam excluídos da base de dados
dataBase.dropna(inplace = True)

In [0]:
#Resetando índices
dataBase.reset_index(drop = True, inplace = True)

In [0]:
#Quantidade de dados perdidos
print("Dados perdidos por eliminação de NaN: " + str(784 - len(dataBase)))

Dados perdidos por eliminação de NaN: 59


In [0]:
#Dividindo base de dados em 3 problemas, com diferentes diagnósticos: Corrosão, Combustão e Contaminação
dataBase["DiagCorr"], dataBase["DiagComb"], dataBase["DiagCont"] = "NaN", "NaN", "NaN"
for i in dataBase.index:
  dataBase.loc[i, "DiagCorr"] = dataBase.loc[i, "Diagnostico"][0] if dataBase.loc[i, "Diagnostico"][0] != " " else "NaN"
  dataBase.loc[i, "DiagComb"] = dataBase.loc[i, "Diagnostico"][1] if dataBase.loc[i, "Diagnostico"][1] != " " else "NaN"
  dataBase.loc[i, "DiagCont"] = dataBase.loc[i, "Diagnostico"][-1] if dataBase.loc[i, "Diagnostico"][-1] != " " else "NaN"

In [0]:
#Eliminando "Código","Tipo","Marca","Data Troca" e "Diagnostico" da base de dados
dataBase.drop(columns = ["Código","Tipo","Marca","Data Troca", "Diagnostico"], inplace = True)

In [0]:
#Gerando variável Kms rodados
dataBase["Kms rodados"] = dataBase["Km Atual"] - dataBase["Km Troca"]

In [0]:
dataBase.head()

,Km Atual,Km Troca,Fe,Cr,Pb,Cu,Sn,Al,Ni,Ag,Si,B,Na,Mg,Ca,Ba,P,Zn,Mo,Ti,V,K,Oxi,Nit,Sul,Ful,Agu,Gli,Dil,DiagCorr,DiagComb,DiagCont,Kms rodados
0,88898.0,14185.0,87.0,15.0,7.0,8.0,8.0,2.0,0.0,0.0,7.0,0.0,196.0,586.0,1305.0,0.0,952.0,1182.0,0.0,0.0,0.0,4.0,"0,03","0,06","0,21","1,13","<0,10","<0,08","<2,00",G,G,G,74713.0
1,100314.0,14805.0,99.0,21.0,10.0,8.0,10.0,2.0,0.0,0.0,7.0,0.0,193.0,558.0,1284.0,0.0,949.0,1177.0,0.0,0.0,0.0,5.0,"0,05","0,09","0,25","1,50","<0,10","<0,08","<2,00",G,Y,G,85509.0
2,436880.0,3637.0,146.0,5.0,13.0,5.0,6.0,15.0,1.0,0.0,15.0,0.0,242.0,676.0,1167.0,0.0,1038.0,1317.0,9.0,0.0,0.0,7.0,"0,03","0,03","0,07","0,45",<0.10,"<0,08","<2,00",G,G,G,433243.0
3,440134.0,7124.0,181.0,6.0,18.0,4.0,7.0,14.0,1.0,0.0,17.0,0.0,269.0,846.0,1386.0,0.0,1233.0,1456.0,11.0,0.0,0.0,7.0,"0,03","0,04","0,09","0,48","<0,10","<0,08","<2,00",G,G,G,433010.0
4,446273.0,5962.0,243.0,10.0,34.0,11.0,9.0,24.0,1.0,0.0,24.0,0.0,261.0,784.0,1235.0,0.0,1146.0,1430.0,19.0,0.0,0.0,5.0,"0,09","0,10","0,22","1,17","<0,10","<0,08","<2,00",Y,G,G,440311.0


In [0]:
dataBase.dropna(inplace = True)

In [0]:
#Guardando a informação de classes, que serão necessárias abaixo na geração de dados artificiais
targetsDiagCorr = dataBase["DiagCorr"]
targetsDiagComb = dataBase["DiagComb"]
targetsDiagCont = dataBase["DiagCont"]

In [0]:
#Eliminando elementos "<" e ">", transformando-os em um valor exato
for j in dataBase.columns:
  for i in dataBase.index:
    if (type(dataBase.loc[i, j]) == str):
      if dataBase.loc[i, j][0] == "<" or dataBase.loc[i, j][0] == ">":
        dataBase.loc[i, j] = float(dataBase.loc[i, j][1:].replace(",","."))
      elif j[:4] != "Diag":
        dataBase.loc[i, j] = float(dataBase.loc[i, j].replace(",","."))

In [0]:
#Transformando todas as colunas que não são de Diagnóstico em float
for j in dataBase.columns:
  if j[:4] != "Diag":
    dataBase[j] = dataBase[j].astype('float64')

In [0]:
dataBase.describe()

,Km Atual,Km Troca,Fe,Cr,Pb,Cu,Sn,Al,Ni,Ag,Si,B,Na,Mg,Ca,Ba,P,Zn,Mo,Ti,V,K,Oxi,Nit,Sul,Ful,Agu,Gli,Dil,Kms rodados
count,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.00000,725.000000,725.000000,725.000000,725.000000,725.00000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000
mean,317346.384828,10903.004138,97.377931,3.628966,11.582069,6.880000,4.183448,8.49931,0.748966,0.044138,10.314483,0.228966,168.22069,471.393103,1676.797241,0.200000,894.340690,1134.522759,6.841379,0.001379,0.001379,4.491034,0.081586,0.096262,0.154331,0.611283,0.106579,0.080207,2.071628,306443.380690
std,121214.743593,4718.952108,48.786892,3.126520,9.176324,6.496127,2.105177,4.05270,0.884420,0.394345,3.985869,2.514596,108.75848,300.390259,704.071919,2.164505,225.798848,292.938359,5.409318,0.037139,0.037139,3.755878,0.042202,0.045985,0.052829,0.312463,0.097391,0.005571,0.615456,120535.190233
min,27752.000000,1619.000000,11.000000,0.000000,0.000000,1.000000,0.000000,1.00000,0.000000,0.000000,4.000000,0.000000,1.00000,15.000000,452.000000,0.000000,268.000000,340.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,-0.000000,0.100000,0.080000,2.000000,984.000000
25%,254186.000000,8742.000000,68.000000,2.000000,7.000000,4.000000,3.000000,6.00000,0.000000,0.000000,8.000000,0.000000,32.00000,98.000000,1125.000000,0.000000,652.000000,821.000000,4.000000,0.000000,0.000000,2.000000,0.060000,0.070000,0.120000,0.410000,0.100000,0.080000,2.000000,244441.000000
50%,291821.000000,9769.000000,85.000000,3.000000,10.000000,5.000000,4.000000,8.00000,1.000000,0.000000,10.000000,0.000000,225.00000,613.000000,1266.000000,0.000000,964.000000,1209.000000,6.000000,0.000000,0.000000,4.000000,0.070000,0.090000,0.150000,0.580000,0.100000,0.080000,2.000000,281657.000000
75%,368163.000000,11450.000000,115.000000,4.000000,14.000000,7.000000,5.000000,10.00000,1.000000,0.000000,12.000000,0.000000,254.00000,718.000000,2473.000000,0.000000,1072.000000,1363.000000,9.000000,0.000000,0.000000,6.000000,0.100000,0.120000,0.190000,0.750000,0.100000,0.080000,2.000000,356651.000000
max,883564.000000,52697.000000,542.000000,31.000000,127.000000,74.000000,27.000000,43.00000,5.000000,7.000000,45.000000,64.000000,400.00000,1037.000000,3828.000000,30.000000,1429.000000,2004.000000,63.000000,1.000000,1.000000,24.000000,0.380000,0.610000,0.430000,2.500000,2.000000,0.230000,10.000000,864040.000000


In [0]:
#Atribuindo valor 0 para "G", 1 para "Y" e 2 para "R", mensurando o status do diagnóstico e permitindo uma análise de correlação com o motor
dataBase.loc[dataBase["DiagCorr"] == "G", "DiagCorr"] = 0 
dataBase.loc[dataBase["DiagCorr"] == "Y", "DiagCorr"] = 1 
dataBase.loc[dataBase["DiagCorr"] == "R", "DiagCorr"] = 2 
dataBase.loc[dataBase["DiagComb"] == "G", "DiagComb"] = 0 
dataBase.loc[dataBase["DiagComb"] == "Y", "DiagComb"] = 1 
dataBase.loc[dataBase["DiagComb"] == "R", "DiagComb"] = 2 
dataBase.loc[dataBase["DiagCont"] == "G", "DiagCont"] = 0 
dataBase.loc[dataBase["DiagCont"] == "Y", "DiagCont"] = 1 
dataBase.loc[dataBase["DiagCont"] == "R", "DiagCont"] = 2 
for j in dataBase.columns:
  dataBase[j] = dataBase[j].astype('float64')

In [0]:
#Analisando correlações entre o diagnóstico e as variáveis
dataBase.corr()[["DiagCorr","DiagComb","DiagCont"]] > 0.5

,DiagCorr,DiagComb,DiagCont
Km Atual,False,False,False
Km Troca,False,False,False
Fe,True,False,False
Cr,False,False,False
Pb,True,False,False
Cu,False,False,False
Sn,False,False,False
Al,True,False,False
Ni,False,False,False
Ag,False,False,False


In [0]:
#Separando diagnósticos da base de dados
dataBase = dataBase.drop(columns = ["DiagCorr","DiagComb","DiagCont"])

In [0]:
from sklearn.decomposition import PCA

In [0]:
#Usando PCA sem diminuir a dimensionalidade, para avaliar o grau de informação que iremos preservar a seguir
pca = PCA(n_components= len(dataBase.columns))
pca.fit(dataBase)

PCA(copy=True, iterated_power='auto', n_components=30, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [0]:
#Grau de informação preservado reduzindo a dimensionalidade de 30 para 20 variáveis
print(sum(pca.explained_variance_ratio_[:20]))

0.9999999999948428


In [0]:
#Considerou-se, assim, 20 como sendo um bom número de variáveis para se trabalhar, reduzindo a seguir a dimensão para 20
pca = PCA(n_components= 20)
dadosReduzidos = pd.DataFrame(pca.fit_transform(dataBase), columns = ["Comp" + str(i) for i in range(20)])

In [0]:
#Normalizando os componentes utilizando Z-Score
for i in dadosReduzidos.columns:
  dadosReduzidos[i] = (dadosReduzidos[i] - dadosReduzidos[i].mean()) / dadosReduzidos[i].std(ddof = 1)

In [0]:
dadosReduzidos.describe()

,Comp0,Comp1,Comp2,Comp3,Comp4,Comp5,Comp6,Comp7,Comp8,Comp9,Comp10,Comp11,Comp12,Comp13,Comp14,Comp15,Comp16,Comp17,Comp18,Comp19
count,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02,7.250000e+02
mean,-3.736475e-16,1.855221e-16,-8.575516e-18,6.500547e-17,1.462432e-16,-2.764072e-17,-3.016744e-17,-4.262874e-17,1.225074e-16,-9.494321e-18,1.195978e-16,-1.424148e-17,1.378208e-17,1.179133e-17,-1.990745e-17,-2.113252e-17,-5.564514e-17,7.381069e-17,2.174506e-17,6.301473e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.457905e+00,-1.904337e+00,-1.551438e+00,-4.338746e+00,-2.263938e+00,-5.413022e+00,-2.778695e+00,-3.199954e+00,-3.740280e+00,-8.323890e+00,-5.418159e+00,-3.632508e+00,-3.028466e+00,-6.249441e+00,-4.722556e+00,-5.565040e+00,-5.002889e+00,-3.207470e+00,-4.288682e+00,-1.842302e+00
25%,-5.181069e-01,-4.924856e-01,-7.705431e-01,-4.615053e-01,-5.740916e-01,-5.893892e-01,-5.932137e-01,-5.707784e-01,-5.079883e-01,-3.709084e-01,-4.483835e-01,-6.643332e-01,-2.098814e-01,-5.006417e-01,-4.056901e-01,-3.138209e-01,-5.264951e-01,-7.023787e-01,-5.746710e-01,-3.506455e-01
50%,-2.066031e-01,-2.194766e-01,-5.992949e-01,-3.239148e-02,-8.877730e-02,-7.381417e-02,-1.139273e-01,-6.347736e-02,-7.011193e-02,-4.721916e-02,1.570840e-01,-1.201375e-01,-3.800686e-02,-4.705095e-02,-7.834292e-02,-1.743759e-02,-5.742928e-02,-7.122729e-03,2.144332e-02,-9.844094e-02
75%,4.198366e-01,1.517785e-01,1.165613e+00,5.341073e-01,3.283679e-01,5.135362e-01,5.151282e-01,5.552246e-01,4.283065e-01,2.761964e-01,6.080247e-01,5.340139e-01,1.391682e-01,4.620669e-01,2.835100e-01,2.697614e-01,4.462939e-01,6.596987e-01,6.326947e-01,1.410620e-01
max,4.649726e+00,8.877927e+00,2.283233e+00,5.685360e+00,8.041789e+00,5.675974e+00,5.319878e+00,4.608465e+00,6.582932e+00,9.319658e+00,6.482839e+00,4.738231e+00,2.340554e+01,7.096232e+00,7.067988e+00,1.153123e+01,8.248242e+00,3.673440e+00,5.555719e+00,1.297424e+01


In [0]:
#Adicionando de volta "diagnostico" aos dados com dimensão reduzida e normalizados
dadosReduzidos["DiagCorr"] = targetsDiagCorr
dadosReduzidos["DiagComb"] = targetsDiagComb
dadosReduzidos["DiagCont"] = targetsDiagCont

In [0]:
dadosReduzidos.head()

,Comp0,Comp1,Comp2,Comp3,Comp4,Comp5,Comp6,Comp7,Comp8,Comp9,Comp10,Comp11,Comp12,Comp13,Comp14,Comp15,Comp16,Comp17,Comp18,Comp19,DiagCorr,DiagComb,DiagCont
0,-1.903728,0.980049,-0.202291,0.291706,-0.298539,-0.132971,0.498385,-0.798861,-0.541295,0.598768,-2.548319,-0.257822,-1.016355,-2.149054,2.478175,-0.320351,0.331915,-0.846083,0.189172,-0.201575,G,G,G
1,-1.811812,1.099386,-0.227143,0.428438,0.125973,0.275974,0.851797,-0.756941,-0.656585,0.271209,-3.335947,0.029260,-1.707017,-3.302968,4.165985,-0.714986,0.146000,-1.354373,-0.883928,-0.369660,G,Y,G
2,1.018868,-1.705004,-0.846403,-0.041736,0.842189,-0.555635,0.244796,-0.142876,-1.128580,-0.010554,0.507689,0.818285,-0.412055,0.577985,0.930307,-0.812919,-0.795720,-0.148252,0.681524,-0.492098,G,G,G
3,1.031488,-0.959793,-0.836798,-1.498708,0.059836,-2.602094,2.039000,-1.002304,-1.533845,-0.122034,0.937490,-0.492297,0.080001,-0.163521,0.710597,-0.740001,0.650265,-0.007123,1.050463,0.080582,G,G,G
4,1.087051,-1.216909,-0.930307,-0.845642,2.089908,-2.257486,1.076195,-1.174943,-0.160109,-0.718905,1.328410,-0.359201,-0.449569,1.381741,2.086331,-2.488323,-0.671658,-1.499391,0.171941,-0.456340,Y,G,G


In [0]:
#Criando base de dados para trabalhar com diferentes diagnósticos
corrDataBase = dadosReduzidos.drop(columns = ["DiagComb","DiagCont"])
print(corrDataBase["DiagCorr"])

0      G
1      G
2      G
3      G
4      Y
      ..
720    G
721    G
722    G
723    G
724    G
Name: DiagCorr, Length: 725, dtype: object


In [0]:
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

In [0]:
#Avaliando total de casos para cada diagnóstico
for i in ["G","Y","R"]:
  if i == "G":
    totalG = sum(corrDataBase["DiagCorr"] == i)
    print("Total de diagnósticos G: " + str(totalG))
  elif i == "Y":
    totalY = sum(corrDataBase["DiagCorr"] == i)
    print("Total de diagnósticos Y: " + str(totalY))
  else:
    totalR = sum(corrDataBase["DiagCorr"] == i)
    print("Total de diagnósticos R: " + str(totalR))

Total de diagnósticos G: 642
Total de diagnósticos Y: 64
Total de diagnósticos R: 19


In [0]:
sm = SMOTE(sampling_strategy = {"G": int(totalG), "Y":int(totalG), "R":int(totalG)}, random_state=2, k_neighbors=3)

In [0]:
dadosGeradosCorr, classesCorr = sm.fit_resample(corrDataBase.drop(columns = "DiagCorr"), corrDataBase["DiagCorr"].values)

In [0]:
corrDataBase = pd.DataFrame(dadosGeradosCorr, columns = corrDataBase.columns[:-1])

In [0]:
print(corrDataBase.shape)

(1926, 20)


In [0]:
print(classesCorr.shape)

(1926,)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

In [0]:
#Transformando cada label "G", "Y" e "R" em 0, 1, e 2, respectivamente
classesCorr[classesCorr == "G"] = 0
classesCorr[classesCorr == "Y"] = 1
classesCorr[classesCorr == "R"] = 2

In [0]:
#Transformando cada label em uma classe
ohe = OneHotEncoder(sparse=False, categories='auto')
classesCorr = pd.DataFrame(ohe.fit_transform(classesCorr.reshape(len(classesCorr), 1)), columns = ["G","Y","R"])
classesCorr.head()

,G,Y,R
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0


In [0]:
corrDataBase.head()

,Comp0,Comp1,Comp2,Comp3,Comp4,Comp5,Comp6,Comp7,Comp8,Comp9,Comp10,Comp11,Comp12,Comp13,Comp14,Comp15,Comp16,Comp17,Comp18,Comp19
0,-1.903728,0.980049,-0.202291,0.291706,-0.298539,-0.132971,0.498385,-0.798861,-0.541295,0.598768,-2.548319,-0.257822,-1.016355,-2.149054,2.478175,-0.320351,0.331915,-0.846083,0.189172,-0.201575
1,-1.811812,1.099386,-0.227143,0.428438,0.125973,0.275974,0.851797,-0.756941,-0.656585,0.271209,-3.335947,0.029260,-1.707017,-3.302968,4.165985,-0.714986,0.146000,-1.354373,-0.883928,-0.369660
2,1.018868,-1.705004,-0.846403,-0.041736,0.842189,-0.555635,0.244796,-0.142876,-1.128580,-0.010554,0.507689,0.818285,-0.412055,0.577985,0.930307,-0.812919,-0.795720,-0.148252,0.681524,-0.492098
3,1.031488,-0.959793,-0.836798,-1.498708,0.059836,-2.602094,2.039000,-1.002304,-1.533845,-0.122034,0.937490,-0.492297,0.080001,-0.163521,0.710597,-0.740001,0.650265,-0.007123,1.050463,0.080582
4,1.087051,-1.216909,-0.930307,-0.845642,2.089908,-2.257486,1.076195,-1.174943,-0.160109,-0.718905,1.328410,-0.359201,-0.449569,1.381741,2.086331,-2.488323,-0.671658,-1.499391,0.171941,-0.456340


In [0]:
def MLP(optimizer='adam', init='random_uniform', units = 20, camadas = 2):    
  #Criando o modelo
  mlp = Sequential()
  #Adicionando a camada de entrada
  mlp.add(Dense(activation = 'tanh', input_dim = units, units = units, kernel_initializer = init, bias_initializer='zeros'))
  #Adicionando as camadas escondidas
  for i in range(camadas):
    mlp.add(Dense(activation = 'tanh', units = units, kernel_initializer = init, bias_initializer='zeros'))
  #Adicionando a camada de saída
  mlp.add(Dense(activation = 'sigmoid', units = 3, kernel_initializer = init, bias_initializer='zeros'))    
  # Compilando o modelo
  mlp.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics= ['accuracy'])
  return mlp

In [0]:
#Separando conjunto de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(corrDataBase, classesCorr, test_size = 0.3, random_state = 9)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size = 0.4, random_state = 9)
Y_valArgMax = np.array([np.argmax(Y_val.loc[i]) for i in Y_val.index])
mlp = MLP()
continueCondition = True
contador = 0
melhorAcuracia = 0
while continueCondition:
  hist = mlp.fit(X_train, Y_train, epochs = 10, verbose = 1)
  resposta = pd.DataFrame(mlp.predict(X_val), columns = ["G","Y","R"])
  resultado = pd.Series(index = resposta.index)
  for i in resposta.index:
    resultado[i] = np.argmax(resposta.loc[i])
  acuracia = sum(Y_valArgMax == np.array(resultado)) / len(resultado)
  if acuracia > melhorAcuracia:
    melhorAcuracia = acuracia
    contador = 0
  else: 
    contador = contador + 1
  if contador == 40:
    continueCondition = False
  if melhorAcuracia > 0.99:
    continueCondition = False  

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


Epoch 1/10
1348/1348 [==============================] - 1s 423us/step - loss: 1.0854 - acc: 0.5445
Epoch 2/10
1348/1348 [==============================] - 0s 50us/step - loss: 0.8874 - acc: 0.5890
Epoch 3/10
1348/1348 [==============================] - 0s 58us/step - loss: 0.6775 - acc: 0.6484
Epoch 4/10
1348/1348 [==============================] - 0s 47us/step - loss: 0.5923 - acc: 0.6617
Epoch 5/10
1348/1348 [==============================] - 0s 52us/step - loss: 0.5522 - acc: 0.6669
Epoch 6/10
1348/1348 [==============================] - 0s 48us/step - loss: 0.5361 - acc: 0.6647
Epoch 7/10
1348/1348 [==============================] - 0s 48us/step - loss: 0.5247 - acc: 0.6550
Epoch 8/10
1348/1348 [==============================] - 0s 46us/step - loss: 0.5160 - acc: 0.6550
Epoch 9/10
1348/1348 [==============================] - 0s 50us/step - loss: 0.5074 - acc: 0.6573
Epoch 10/10
1348/1348 [==============================] - 0s 53us/step - loss: 0.5026 - acc: 0.6580
Epoch 1/10
1348/13

In [0]:
y_predicted = pd.DataFrame(mlp.predict(X_test), columns = ["G","Y","R"])
y_predicted = np.array([np.argmax(y_predicted.loc[i]) for i in y_predicted.index])
resultado = pd.Series(index = Y_test.index)
for i in Y_test.index:
  resultado[i] = np.argmax(Y_test.loc[i])

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [0]:
pd.DataFrame(confusion_matrix(y_predicted, resultado), index = ["G","Y","R"], columns = ["G","Y","R"])

,G,Y,R
G,111,0,6
Y,1,109,0
R,5,0,114


In [0]:
combDataBase = dadosReduzidos.drop(columns = ["DiagCorr","DiagCont"])

for i in ["G","Y","R"]:
  if i == "G":
    totalG = sum(combDataBase["DiagComb"] == i)
    print("Total de diagnósticos G: " + str(totalG))
  elif i == "Y":
    totalY = sum(combDataBase["DiagComb"] == i)
    print("Total de diagnósticos Y: " + str(totalY))
  else:
    totalR = sum(combDataBase["DiagComb"] == i)
    print("Total de diagnósticos R: " + str(totalR))

Total de diagnósticos G: 706
Total de diagnósticos Y: 14
Total de diagnósticos R: 4


In [0]:
sm = SMOTE(sampling_strategy = {"G": int(totalG), "Y":int(totalG), "R":int(totalG)}, random_state=2, k_neighbors=3)

In [0]:
dadosGeradosComb, classesComb = sm.fit_resample(combDataBase.drop(columns = "DiagComb"), combDataBase["DiagComb"].values)

In [0]:
combDataBase = pd.DataFrame(dadosGeradosComb, columns = combDataBase.columns[:-1])

In [0]:
print(combDataBase.shape)

(2119, 20)


In [0]:
print(classesComb.shape)

(2119,)


In [0]:
#Transformando cada label "G", "Y" e "R" em 0, 1, e 2, respectivamente
classesComb[classesComb == "G"] = 0
classesComb[classesComb == "Y"] = 1
classesComb[classesComb == "R"] = 2
classesComb[classesComb == "NaN"] = np.nan

In [0]:
combDataBase["Classes"] = classesComb
combDataBase.dropna(inplace = True)
classesComb = combDataBase["Classes"].values
combDataBase.drop(columns = "Classes", inplace = True)

In [0]:
#Transformando cada label em uma classe
ohe = OneHotEncoder(sparse=False, categories='auto')
classesComb = pd.DataFrame(ohe.fit_transform(classesComb.reshape(len(classesComb), 1)), columns = ["G","Y","R"])
classesComb.head()

,G,Y,R
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [0]:
combDataBase.head()

,Comp0,Comp1,Comp2,Comp3,Comp4,Comp5,Comp6,Comp7,Comp8,Comp9,Comp10,Comp11,Comp12,Comp13,Comp14,Comp15,Comp16,Comp17,Comp18,Comp19
0,-1.903728,0.980049,-0.202291,0.291706,-0.298539,-0.132971,0.498385,-0.798861,-0.541295,0.598768,-2.548319,-0.257822,-1.016355,-2.149054,2.478175,-0.320351,0.331915,-0.846083,0.189172,-0.201575
1,-1.811812,1.099386,-0.227143,0.428438,0.125973,0.275974,0.851797,-0.756941,-0.656585,0.271209,-3.335947,0.029260,-1.707017,-3.302968,4.165985,-0.714986,0.146000,-1.354373,-0.883928,-0.369660
2,1.018868,-1.705004,-0.846403,-0.041736,0.842189,-0.555635,0.244796,-0.142876,-1.128580,-0.010554,0.507689,0.818285,-0.412055,0.577985,0.930307,-0.812919,-0.795720,-0.148252,0.681524,-0.492098
3,1.031488,-0.959793,-0.836798,-1.498708,0.059836,-2.602094,2.039000,-1.002304,-1.533845,-0.122034,0.937490,-0.492297,0.080001,-0.163521,0.710597,-0.740001,0.650265,-0.007123,1.050463,0.080582
4,1.087051,-1.216909,-0.930307,-0.845642,2.089908,-2.257486,1.076195,-1.174943,-0.160109,-0.718905,1.328410,-0.359201,-0.449569,1.381741,2.086331,-2.488323,-0.671658,-1.499391,0.171941,-0.456340


In [0]:
#Separando conjunto de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(combDataBase, classesComb, test_size = 0.3, random_state = 9)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size = 0.4, random_state = 9)
Y_valArgMax = np.array([np.argmax(Y_val.loc[i]) for i in Y_val.index])
mlp = MLP()
continueCondition = True
contador = 0
melhorAcuracia = 0
while continueCondition:
  hist = mlp.fit(X_train, Y_train, epochs = 10, verbose = 1)
  resposta = pd.DataFrame(mlp.predict(X_val), columns = ["G","Y","R"])
  resultado = pd.Series(index = resposta.index)
  for i in resposta.index:
    resultado[i] = np.argmax(resposta.loc[i])
  acuracia = sum(Y_valArgMax == np.array(resultado)) / len(resultado)
  if acuracia > melhorAcuracia:
    melhorAcuracia = acuracia
    contador = 0
  else: 
    contador = contador + 1
  if contador == 40:
    continueCondition = False
  if melhorAcuracia > 0.98:
    continueCondition = False    

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


Epoch 1/10
1482/1482 [==============================] - 1s 379us/step - loss: 1.0834 - acc: 0.5344
Epoch 2/10
1482/1482 [==============================] - 0s 44us/step - loss: 0.9050 - acc: 0.6140
Epoch 3/10
1482/1482 [==============================] - 0s 47us/step - loss: 0.7431 - acc: 0.6457
Epoch 4/10
1482/1482 [==============================] - 0s 46us/step - loss: 0.6690 - acc: 0.6619
Epoch 5/10
1482/1482 [==============================] - 0s 45us/step - loss: 0.6149 - acc: 0.6572
Epoch 6/10
1482/1482 [==============================] - 0s 45us/step - loss: 0.5740 - acc: 0.6559
Epoch 7/10
1482/1482 [==============================] - 0s 45us/step - loss: 0.5532 - acc: 0.6640
Epoch 8/10
1482/1482 [==============================] - 0s 44us/step - loss: 0.5443 - acc: 0.6903
Epoch 9/10
1482/1482 [==============================] - 0s 53us/step - loss: 0.5374 - acc: 0.7355
Epoch 10/10
1482/1482 [==============================] - 0s 49us/step - loss: 0.5335 - acc: 0.7341
Epoch 1/10
1482/14

In [0]:
y_predicted = pd.DataFrame(mlp.predict(X_test), columns = ["G","Y","R"])
y_predicted = np.array([np.argmax(y_predicted.loc[i]) for i in y_predicted.index])
resultado = pd.Series(index = Y_test.index)
for i in Y_test.index:
  resultado[i] = np.argmax(Y_test.loc[i])

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [0]:
pd.DataFrame(confusion_matrix(y_predicted, resultado), index = ["G","Y","R"], columns = ["G","Y","R"])

,G,Y,R
G,131,0,0
Y,0,121,0
R,5,0,124


In [0]:
contDataBase = dadosReduzidos.drop(columns = ["DiagCorr","DiagComb"])

for i in ["G","Y","R"]:
  if i == "G":
    totalG = sum(contDataBase["DiagCont"] == i)
    print("Total de diagnósticos G: " + str(totalG))
  elif i == "Y":
    totalY = sum(contDataBase["DiagCont"] == i)
    print("Total de diagnósticos Y: " + str(totalY))
  else:
    totalR = sum(contDataBase["DiagCont"] == i)
    print("Total de diagnósticos R: " + str(totalR))

Total de diagnósticos G: 695
Total de diagnósticos Y: 8
Total de diagnósticos R: 22


In [0]:
sm = SMOTE(sampling_strategy = {"G": int(totalG), "Y":int(totalG), "R":int(totalG)}, random_state=2, k_neighbors=3)

In [0]:
dadosGeradosCont, classesCont = sm.fit_resample(contDataBase.drop(columns = "DiagCont"), contDataBase["DiagCont"].values)

In [0]:
contDataBase = pd.DataFrame(dadosGeradosCont, columns = contDataBase.columns[:-1])

In [0]:
#Transformando cada label "G", "Y" e "R" em 0, 1, e 2, respectivamente
classesCont[classesCont == "G"] = 0
classesCont[classesCont == "Y"] = 1
classesCont[classesCont == "R"] = 2

In [0]:
contDataBase["Classes"] = classesCont
contDataBase.dropna(inplace = True)
classesCont = contDataBase["Classes"].values
contDataBase.drop(columns = "Classes", inplace = True)

In [0]:
#Transformando cada label em uma classe
ohe = OneHotEncoder(sparse=False, categories='auto')
classesCont = pd.DataFrame(ohe.fit_transform(classesCont.reshape(len(classesCont), 1)), columns = ["G","Y","R"])
classesCont.head()

,G,Y,R
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [0]:
contDataBase.head()

,Comp0,Comp1,Comp2,Comp3,Comp4,Comp5,Comp6,Comp7,Comp8,Comp9,Comp10,Comp11,Comp12,Comp13,Comp14,Comp15,Comp16,Comp17,Comp18,Comp19
0,-1.903728,0.980049,-0.202291,0.291706,-0.298539,-0.132971,0.498385,-0.798861,-0.541295,0.598768,-2.548319,-0.257822,-1.016355,-2.149054,2.478175,-0.320351,0.331915,-0.846083,0.189172,-0.201575
1,-1.811812,1.099386,-0.227143,0.428438,0.125973,0.275974,0.851797,-0.756941,-0.656585,0.271209,-3.335947,0.029260,-1.707017,-3.302968,4.165985,-0.714986,0.146000,-1.354373,-0.883928,-0.369660
2,1.018868,-1.705004,-0.846403,-0.041736,0.842189,-0.555635,0.244796,-0.142876,-1.128580,-0.010554,0.507689,0.818285,-0.412055,0.577985,0.930307,-0.812919,-0.795720,-0.148252,0.681524,-0.492098
3,1.031488,-0.959793,-0.836798,-1.498708,0.059836,-2.602094,2.039000,-1.002304,-1.533845,-0.122034,0.937490,-0.492297,0.080001,-0.163521,0.710597,-0.740001,0.650265,-0.007123,1.050463,0.080582
4,1.087051,-1.216909,-0.930307,-0.845642,2.089908,-2.257486,1.076195,-1.174943,-0.160109,-0.718905,1.328410,-0.359201,-0.449569,1.381741,2.086331,-2.488323,-0.671658,-1.499391,0.171941,-0.456340


In [0]:
#Separando conjunto de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(contDataBase, classesCont, test_size = 0.3, random_state = 9)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size = 0.4, random_state = 9)
Y_valArgMax = np.array([np.argmax(Y_val.loc[i]) for i in Y_val.index])
mlp = MLP()
continueCondition = True
contador = 0
melhorAcuracia = 0
while continueCondition:
  hist = mlp.fit(X_train, Y_train, epochs = 10, verbose = 1)
  resposta = pd.DataFrame(mlp.predict(X_val), columns = ["G","Y","R"])
  resultado = pd.Series(index = resposta.index)
  for i in resposta.index:
    resultado[i] = np.argmax(resposta.loc[i])
  acuracia = sum(Y_valArgMax == np.array(resultado)) / len(resultado)
  if acuracia > melhorAcuracia:
    melhorAcuracia = acuracia
    contador = 0
  else: 
    contador = contador + 1
  if contador == 40:
    continueCondition = False
  if melhorAcuracia > 0.98:
    continueCondition = False    

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


Epoch 1/10
1459/1459 [==============================] - 1s 625us/step - loss: 1.0788 - acc: 0.5195
Epoch 2/10
1459/1459 [==============================] - 0s 50us/step - loss: 0.8077 - acc: 0.6470
Epoch 3/10
1459/1459 [==============================] - 0s 55us/step - loss: 0.6034 - acc: 0.6525
Epoch 4/10
1459/1459 [==============================] - 0s 56us/step - loss: 0.5419 - acc: 0.6621
Epoch 5/10
1459/1459 [==============================] - 0s 60us/step - loss: 0.5215 - acc: 0.6648
Epoch 6/10
1459/1459 [==============================] - 0s 49us/step - loss: 0.5096 - acc: 0.6628
Epoch 7/10
1459/1459 [==============================] - 0s 50us/step - loss: 0.5012 - acc: 0.6635
Epoch 8/10
1459/1459 [==============================] - 0s 57us/step - loss: 0.4967 - acc: 0.6635
Epoch 9/10
1459/1459 [==============================] - 0s 49us/step - loss: 0.4928 - acc: 0.6628
Epoch 10/10
1459/1459 [==============================] - 0s 51us/step - loss: 0.4906 - acc: 0.6628
Epoch 1/10
1459/14

In [0]:
y_predicted = pd.DataFrame(mlp.predict(X_test), columns = ["G","Y","R"])
y_predicted = np.array([np.argmax(y_predicted.loc[i]) for i in y_predicted.index])
resultado = pd.Series(index = Y_test.index)
for i in Y_test.index:
  resultado[i] = np.argmax(Y_test.loc[i])

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [0]:
pd.DataFrame(confusion_matrix(y_predicted, resultado), index = ["G","Y","R"], columns = ["G","Y","R"])

,G,Y,R
G,118,0,0
Y,3,120,0
R,3,0,131
